In [ ]:
import psycopg2
import pandas as pd
import requests
import csv
import yfinance as yf


print(psycopg2.__version__)
print(pd.__version__)
print(requests.__version__)
print(csv.__version__)
print(yf.__version__)


2.9.11 (dt dec pq3 ext lo64)
2.3.3
2.32.5
1.0
0.2.66


In [ ]:
import requests
import csv
import psycopg2
import pandas as pd
import yfinance as yf

# Database connection parameters
db_host = "localhost"
db_name = "bootcamp_2508_final_project"
db_user = "postgres"
db_password = "admin1234"

# Connect to the PostgreSQL database
conn = psycopg2.connect(
    host=db_host,
    database=db_name,
    user=db_user,
    password=db_password
)

# URL of the CSV file
url = "https://raw.githubusercontent.com/datasets/s-and-p-500-companies/refs/heads/main/data/constituents.csv"


# Fetch the CSV data
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
   # Decode the content and split into lines
   content = response.content.decode('utf-8').splitlines()
    
# Read the CSV data
reader = csv.reader(content)
next(reader)  # Skip the header row

# Extract symbols and write to a .txt file
with open("sp500_symbols.txt", "w") as file:
  for row in reader:
      symbol = row[0]  # The first column contains the symbol
      file.write(symbol + "\n")
      print(f"{symbol} have been saved to sp500_symbols.txt.")
  else:
      print("Failed to fetch data:", response.status_code)

In [ ]:

import psycopg2

# Database connection parameters
db_host = "localhost"
db_name = "bootcamp_2508_final_project"
db_user = "postgres"
db_password = "admin1234"

# Connect to the PostgreSQL database
conn = psycopg2.connect(
    host=db_host,
    database=db_name,
    user=db_user,
    password=db_password
)

# Create a cursor object
cursor = conn.cursor()

# Create table
create_table_query = """
CREATE TABLE IF NOT EXISTS sp500_symbols (
    symbol VARCHAR(10) primary key NOT NULL UNIQUE,
);
"""
cursor.execute(create_table_query)
conn.commit()

# Open the file and read symbols
with open("sp500_symbols.txt", "r") as file:
    symbols = file.readlines()

# Insert symbols into the database
for symbol in symbols:
    symbol = symbol.strip()  # Remove any whitespace/newline characters
    try:
        cursor.execute("INSERT INTO sp500_symbols (symbol) VALUES (%s) ON CONFLICT (symbol) DO NOTHING;", (symbol,))
    except Exception as e:
        print(f"Error inserting {symbol}: {e}")

# Commit changes and close the connection
conn.commit()
cursor.close()
conn.close()

print("S&P 500 symbols have been stored in the PostgreSQL database.")

S&P 500 symbols have been stored in the PostgreSQL database.


In [2]:
import psycopg2
import csv
import re
from datetime import datetime
import requests

# Database connection parameters
db_host = "localhost"
db_name = "bootcamp_2508_final_project"
db_user = "postgres"
db_password = "admin1234"

# Connect to the PostgreSQL database
conn = psycopg2.connect(
    host=db_host,
    database=db_name,
    user=db_user,
    password=db_password
)

# Create a cursor object
cursor = conn.cursor()

# Create table
create_table_query = """
create table IF NOT EXISTS sp500_info (
    symbol VARCHAR(10) primary key NOT NULL UNIQUE,
    security VARCHAR(255) NOT NULL,
    gics_sector VARCHAR(255) NOT NULL,
    gics_sub_industry VARCHAR(255) NOT NULL,
    headquarters_location VARCHAR(255) NOT NULL,
    date_added DATE NOT NULL,
    cik VARCHAR(20) NOT NULL,
    founded TEXT NOT NULL
);
"""
cursor.execute(create_table_query)
conn.commit()

# Load CSV data from URL
url = 'https://raw.githubusercontent.com/datasets/s-and-p-500-companies/refs/heads/main/data/constituents.csv'
# Fetch the CSV data
response = requests.get(url)


# Check if the request was successful
if response.status_code == 200:
    # Open a CSV file and read data
    # Decode the content and split into lines
    lines = response.content.decode('utf-8').splitlines()
    # Read the CSV data
    reader = csv.reader(lines)
    header = next(reader)  # Get the header row

    # Extract symbols and write to a .txt file
    with open("sp500_information.txt", "w", newline='', encoding='utf-8') as txt_file:
        # Optional: write the header as a comment line
        txt_file.write("# " + ",".join(header) + "\n")

        for row in reader:
            # Write the full row as a CSV line (comma-separated)
            txt_file.write(",".join(row) + "\n")

    print("All rows & columns have been saved to sp500_information.txt.")
else:
    print("Failed to fetch data:", response.status_code)

# Re-open the same data for DB insertion (re-use the same reader)
    # Reset the reader because we already consumed it above
lines = response.content.decode('utf-8').splitlines()
reader = csv.reader(lines)
next(reader)   # skip header again

# Insert data into the database
for row in reader:
    # Convert date from string to date object
    date_added = datetime.strptime(row[5], '%Y-%m-%d').date()
    
    # Clean the founded value
    founded_cleaned = re.sub(r'\s*\(.*\)', '', row[7])  # Remove anything in parentheses

    cursor.execute("""
        INSERT INTO sp500_info (symbol, security, gics_sector, gics_sub_industry, headquarters_location, date_added, cik, founded)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (symbol) DO NOTHING;
    """, (row[0], row[1], row[2], row[3], row[4], date_added, row[6], founded_cleaned))

# Commit changes and close the connection
conn.commit()
cursor.close()
conn.close()

print("S&P 500 companies have been stored in the PostgreSQL database.")

All rows & columns have been saved to sp500_information.txt.
S&P 500 companies have been stored in the PostgreSQL database.


In [6]:
#!/usr/bin/env python3
"""
save_all_finnhub_to_one_json.py
-------------------------------
Read sp500_symbols.txt → fetch Finnhub profile2 → save ALL to ONE JSON file
"""

import json
import time
import logging
from pathlib import Path
from typing import Dict, Any, List

import requests
from datetime import datetime


# ----------------------------------------------------------------------
# CONFIGURATION
# ----------------------------------------------------------------------
FINNHUB_TOKEN = "d3rfce1r01qopgh8ajv0d3rfce1r01qopgh8ajvg"
FINNHUB_URL   = "https://finnhub.io/api/v1/stock/profile2"

SYMBOLS_TXT   = "sp500_symbols.txt"

# OUTPUT PATH (Windows style)
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
OUTPUT_PATH = Path(f"C:/github/Bootcamp-Final-Project/python/sp500_finnhub_profiles_{timestamp}.json")

REQUEST_DELAY = 1.1  # Safe for free tier

# ----------------------------------------------------------------------
# Logging
# ----------------------------------------------------------------------
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    handlers=[
        logging.FileHandler("fetch_one_json.log"),
        logging.StreamHandler()
    ]
)
log = logging.getLogger(__name__)

# ----------------------------------------------------------------------
# Helpers
# ----------------------------------------------------------------------
def load_symbols() -> List[str]:
    p = Path(SYMBOLS_TXT)
    if not p.is_file():
        log.error(f"{SYMBOLS_TXT} not found in current directory!")
        return []
    syms = [ln.strip().upper() for ln in p.open() if ln.strip() and not ln.startswith("#")]
    log.info(f"Loaded {len(syms)} symbols")
    return syms

def finnhub_profile(symbol: str) -> Dict[str, Any]:
    sym = symbol.replace(".", "-")
    url = f"{FINNHUB_URL}?symbol={sym}&token={FINNHUB_TOKEN}"
    try:
        r = requests.get(url, timeout=12)
        if r.status_code == 200:
            data = r.json()
            return data if isinstance(data, dict) else {}
        if r.status_code == 429:
            log.warning(f"Rate limit – sleep 15s for {symbol}")
            time.sleep(15)
            return finnhub_profile(symbol)
        log.warning(f"HTTP {r.status_code} for {symbol}")
        return {}
    except Exception as e:
        log.error(f"Error {symbol}: {e}")
        return {}

# ----------------------------------------------------------------------
# Main
# ----------------------------------------------------------------------
def main() -> None:
    symbols = load_symbols()
    if not symbols:
        return

    # Ensure parent directory exists
    OUTPUT_PATH.parent.mkdir(parents=True, exist_ok=True)

    all_data = {}

    log.info(f"Starting fetch → saving to {OUTPUT_PATH}")

    for i, sym in enumerate(symbols, 1):
        log.info(f"[{i}/{len(symbols)}] Fetching {sym}")
        profile = finnhub_profile(sym)
        all_data[sym] = profile
        time.sleep(REQUEST_DELAY)

    # Save all at once
    try:
        with OUTPUT_PATH.open("w", encoding="utf-8") as f:
            json.dump(all_data, f, indent=2, ensure_ascii=False)
        log.info(f"SUCCESS: All {len(all_data)} profiles saved to:\n   {OUTPUT_PATH}")
    except Exception as e:
        log.error(f"Failed to write file: {e}")

if __name__ == "__main__":
    main()

2025-10-30 10:27:55,323 | INFO | Loaded 503 symbols
2025-10-30 10:27:55,325 | INFO | Starting fetch → saving to C:\github\Bootcamp-Final-Project\python\sp500_finnhub_profiles.json
2025-10-30 10:27:55,329 | INFO | [1/503] Fetching MMM
2025-10-30 10:27:57,303 | INFO | [2/503] Fetching AOS
2025-10-30 10:27:59,007 | INFO | [3/503] Fetching ABT
2025-10-30 10:28:00,709 | INFO | [4/503] Fetching ABBV
2025-10-30 10:28:02,459 | INFO | [5/503] Fetching ACN
2025-10-30 10:28:04,365 | INFO | [6/503] Fetching ADBE
2025-10-30 10:28:06,146 | INFO | [7/503] Fetching AMD
2025-10-30 10:28:07,948 | INFO | [8/503] Fetching AES
2025-10-30 10:28:09,727 | INFO | [9/503] Fetching AFL
2025-10-30 10:28:11,532 | INFO | [10/503] Fetching A
2025-10-30 10:28:13,271 | INFO | [11/503] Fetching APD
2025-10-30 10:28:14,952 | INFO | [12/503] Fetching ABNB
2025-10-30 10:28:16,870 | INFO | [13/503] Fetching AKAM
2025-10-30 10:28:18,799 | INFO | [14/503] Fetching ALB
2025-10-30 10:28:20,650 | INFO | [15/503] Fetching ARE
20

In [14]:
#!/usr/bin/env python3
"""
load_json_to_postgres_clarity.py
-------------------------------
Load sp500_finnhub_profiles_*.json → PostgreSQL
Keep original numbers, just rename columns to show units
"""

import json
import logging
from pathlib import Path
from typing import Dict, Any
import psycopg2
import psycopg2.extras

# ----------------------------------------------------------------------
# CONFIGURATION
# ----------------------------------------------------------------------
JSON_GLOB = "C:/github/Bootcamp-Final-Project/python/sp500_finnhub_profiles.json"
DB_CONFIG = {
    "host": "localhost",
    "database": "bootcamp_2508_final_project",
    "user": "postgres",
    "password": "admin1234"
}

# ----------------------------------------------------------------------
# Logging
# ----------------------------------------------------------------------
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    handlers=[
        logging.FileHandler("load_json_clarity.log"),
        logging.StreamHandler()
    ]
)
log = logging.getLogger(__name__)

# ----------------------------------------------------------------------
# Create table with CLEAR column names
# ----------------------------------------------------------------------
def ensure_table(cur) -> None:
    cur.execute("""
    CREATE TABLE IF NOT EXISTS sp500_finnhub_profiles (
        symbol                     VARCHAR(10) PRIMARY KEY,
        country                    VARCHAR(100),
        currency                   VARCHAR(10),
        estimate_currency          VARCHAR(10),
        exchange                   VARCHAR(255),
        main_industry              VARCHAR(255),

        -- === UNITS CLEARLY STATED IN COLUMN NAME ===
        floating_share_millions    NUMERIC,   -- in millions (e.g., 14547.72 = 14.5B shares)
        market_cap_usd_millions    NUMERIC,   -- in USD millions (e.g., 4002453 = $4T)
        share_outstanding_millions NUMERIC,   -- in millions

        ipo_date                   DATE,
        logo                       TEXT,
        name                       VARCHAR(255),
        phone                      VARCHAR(50),
        ticker                     VARCHAR(10),
        weburl                     TEXT,
        raw_json                   JSONB      -- full original Finnhub response
    );
    """)
    log.info("Table sp500_finnhub_profile created with clear unit column names")

# ----------------------------------------------------------------------
# Upsert: map Finnhub fields → renamed columns (NO SCALING!)
# ----------------------------------------------------------------------
def upsert(cur, symbol: str, data: Dict[str, Any]) -> None:
    cur.execute("""
    INSERT INTO sp500_finnhub_profiles (
        symbol, country, currency, estimate_currency, exchange,
        main_industry,
        floating_share_millions, market_cap_usd_millions, share_outstanding_millions,
        ipo_date, logo, name, phone, ticker, weburl, raw_json
    ) VALUES (
        %(symbol)s, %(country)s, %(currency)s, %(estimateCurrency)s,
        %(exchange)s, %(finnhubIndustry)s,
        %(floatingShare)s, %(marketCapitalization)s, %(shareOutstanding)s,
        %(ipo)s::date, %(logo)s, %(name)s, %(phone)s,
        %(ticker)s, %(weburl)s, %(raw_json)s
    )
    ON CONFLICT (symbol) DO UPDATE SET
        country                    = EXCLUDED.country,
        currency                   = EXCLUDED.currency,
        estimate_currency          = EXCLUDED.estimate_currency,
        exchange                   = EXCLUDED.exchange,
        main_industry              = EXCLUDED.main_industry,
        floating_share_millions    = EXCLUDED.floating_share_millions,
        market_cap_usd_millions    = EXCLUDED.market_cap_usd_millions,
        share_outstanding_millions = EXCLUDED.share_outstanding_millions,
        ipo_date                   = EXCLUDED.ipo_date,
        logo                       = EXCLUDED.logo,
        name                       = EXCLUDED.name,
        phone                      = EXCLUDED.phone,
        ticker                     = EXCLUDED.ticker,
        weburl                     = EXCLUDED.weburl,
        raw_json                   = EXCLUDED.raw_json;
    """, {
        "symbol": symbol,
        "country": data.get("country"),
        "currency": data.get("currency"),
        "estimateCurrency": data.get("estimateCurrency"),
        "exchange": data.get("exchange"),
        "finnhubIndustry": data.get("finnhubIndustry"),

        # Direct mapping — NO division!
        "floatingShare": data.get("floatingShare"),
        "marketCapitalization": data.get("marketCapitalization"),
        "shareOutstanding": data.get("shareOutstanding"),

        "ipo": data.get("ipo"),
        "logo": data.get("logo"),
        "name": data.get("name"),
        "phone": data.get("phone"),
        "ticker": data.get("ticker"),
        "weburl": data.get("weburl"),
        "raw_json": psycopg2.extras.Json(data)
    })

# ----------------------------------------------------------------------
# Main
# ----------------------------------------------------------------------
def main() -> None:
    json_files = list(Path("C:/github/Bootcamp-Final-Project/python").glob("sp500_finnhub_profiles.json"))
    if not json_files:
        log.error("No JSON file found!")
        return
    json_path = max(json_files, key=lambda p: p.stat().st_mtime)
    log.info(f"Loading from: {json_path}")

    try:
        with json_path.open("r", encoding="utf-8") as f:
            all_data = json.load(f)
        log.info(f"Loaded {len(all_data)} profiles")
    except Exception as e:
        log.error(f"JSON read error: {e}")
        return

    conn = None
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        cur = conn.cursor()
        ensure_table(cur)

        count = 0
        for symbol, profile in all_data.items():
            if not profile:
                log.warning(f"Skipping empty: {symbol}")
                continue
            upsert(cur, symbol, profile)
            count += 1
            if count % 50 == 0:
                conn.commit()
                log.info(f"Committed {count} rows")

        conn.commit()
        log.info(f"SUCCESS: {count} records loaded with clear column names")
        cur.close()
    except Exception as e:
        log.error(f"DB error: {e}")
        if conn:
            conn.rollback()
    finally:
        if conn:
            conn.close()

if __name__ == "__main__":
    main()

2025-10-30 12:19:16,014 | INFO | Loading from: C:\github\Bootcamp-Final-Project\python\sp500_finnhub_profiles.json
2025-10-30 12:19:16,022 | INFO | Loaded 503 profiles
2025-10-30 12:19:16,126 | INFO | Table sp500_finnhub_profile created with clear unit column names
2025-10-30 12:19:16,178 | INFO | Committed 50 rows
2025-10-30 12:19:16,217 | WARNING | Skipping empty: COR
2025-10-30 12:19:16,226 | INFO | Committed 100 rows
2025-10-30 12:19:16,246 | WARNING | Skipping empty: CEG
2025-10-30 12:19:16,247 | WARNING | Skipping empty: COO
2025-10-30 12:19:16,248 | WARNING | Skipping empty: CPRT
2025-10-30 12:19:16,277 | INFO | Committed 150 rows
2025-10-30 12:19:16,313 | WARNING | Skipping empty: FIS
2025-10-30 12:19:16,315 | WARNING | Skipping empty: FITB
2025-10-30 12:19:16,326 | INFO | Committed 200 rows
2025-10-30 12:19:16,335 | WARNING | Skipping empty: GIS
2025-10-30 12:19:16,348 | WARNING | Skipping empty: HSIC
2025-10-30 12:19:16,355 | WARNING | Skipping empty: HST
2025-10-30 12:19:16,

In [15]:
#!/usr/bin/env python3
"""
refetch_failed_symbols.py
-------------------------
1. Scan JSON for empty/missing data
2. Refetch only failed symbols from Finnhub
3. Merge back to original JSON
4. (Optional) Re-load to PostgreSQL
"""

import json
import time
import logging
from pathlib import Path
from typing import Dict, Any, List

import requests

# ----------------------------------------------------------------------
# CONFIGURATION
# ----------------------------------------------------------------------
FINNHUB_TOKEN = "d3rfce1r01qopgh8ajv0d3rfce1r01qopgh8ajvg"  # Your API key
FINNHUB_URL = "https://finnhub.io/api/v1/stock/profile2"

JSON_GLOB = "C:/github/Bootcamp-Final-Project/python/sp500_finnhub_profiles.json"  # Your JSON pattern
FAILED_TXT = "failed_finnhub_profiles.txt"  # Output: list of symbols to refetch

REQUEST_DELAY = 1.1  # Safe rate limit
UPDATE_JSON = True   # Set False to skip JSON update
RELOAD_DB = True     # Set False to skip DB reload

DB_CONFIG = {  # PostgreSQL (if reloading)
    "host": "localhost",
    "database": "bootcamp_2508_final_project",
    "user": "postgres",
    "password": "admin1234"
}

# ----------------------------------------------------------------------
# Logging
# ----------------------------------------------------------------------
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    handlers=[logging.FileHandler("refetch_failed.log"), logging.StreamHandler()]
)
log = logging.getLogger(__name__)

# ----------------------------------------------------------------------
# Helpers
# ----------------------------------------------------------------------
def load_json() -> tuple[Dict[str, Any], Path]:
    """Load the latest JSON file."""
    json_files = list(Path("C://github/Bootcamp-Final-Project/python").glob("sp500_finnhub_profiles.json"))
    if not json_files:
        log.error("No JSON file found!")
        return {}, Path()
    json_path = max(json_files, key=lambda p: p.stat().st_mtime)
    with json_path.open("r", encoding="utf-8") as f:
        data = json.load(f)
    log.info(f"Loaded {len(data)} profiles from {json_path}")
    return data, json_path

def find_failures(data: Dict[str, Any]) -> List[str]:
    """Find symbols with empty or incomplete profiles."""
    failures = []
    for sym, prof in data.items():
        if (not prof or                     # Empty dict
            not prof.get("name") or         # No name
            "marketCapitalization" not in prof or  # No market cap
            not prof.get("country")):       # No country
            failures.append(sym)
    log.info(f"Found {len(failures)} failures: {failures}")
    return failures

def save_failed_list(failures: List[str]) -> None:
    """Save failures to TXT for manual use."""
    with open(FAILED_TXT, "w") as f:
        f.write("\n".join(failures))
    log.info(f"Saved failures to {FAILED_TXT}")

def finnhub_profile(symbol: str) -> Dict[str, Any]:
    """Fetch one profile (with retry)."""
    sym = symbol.replace(".", "-")
    url = f"{FINNHUB_URL}?symbol={sym}&token={FINNHUB_TOKEN}"
    try:
        r = requests.get(url, timeout=12)
        if r.status_code == 200:
            data = r.json()
            return data if isinstance(data, dict) else {}
        if r.status_code == 429:
            log.warning(f"Rate limit for {symbol} – sleep 15s")
            time.sleep(15)
            return finnhub_profile(symbol)
        log.warning(f"HTTP {r.status_code} for {symbol}")
        return {}
    except Exception as e:
        log.error(f"Error {symbol}: {e}")
        return {}

# ----------------------------------------------------------------------
# Update JSON
# ----------------------------------------------------------------------
def update_json(data: Dict[str, Any], json_path: Path, new_profiles: Dict[str, Any]) -> None:
    """Merge new data into original JSON."""
    for sym, prof in new_profiles.items():
        data[sym] = prof
    with json_path.open("w", encoding="utf-8") as f:
        json.dump(data, f, indent=2, ensure_ascii=False)
    log.info(f"Updated JSON with {len(new_profiles)} new profiles")

# ----------------------------------------------------------------------
# Reload to DB (optional)
# ----------------------------------------------------------------------
def reload_to_db(data: Dict[str, Any]) -> None:
    """Re-run upsert for all data (or just failures)."""
    import psycopg2
    import psycopg2.extras

    def ensure_table(cur):
        cur.execute("""
        CREATE TABLE IF NOT EXISTS sp500_finnhub (
            symbol                     VARCHAR(10) PRIMARY KEY,
            country                    VARCHAR(100),
            currency                   VARCHAR(10),
            estimate_currency          VARCHAR(10),
            exchange                   VARCHAR(255),
            main_industry              VARCHAR(255),
            floating_share_millions    NUMERIC,
            market_cap_usd_millions    NUMERIC,
            share_outstanding_millions NUMERIC,
            ipo_date                   DATE,
            logo                       TEXT,
            name                       VARCHAR(255),
            phone                      VARCHAR(50),
            ticker                     VARCHAR(10),
            weburl                     TEXT,
            raw_json                   JSONB
        );
        """)

    def upsert(cur, sym: str, prof: Dict[str, Any]):
        cur.execute("""
        INSERT INTO sp500_finnhub (
            symbol, country, currency, estimate_currency, exchange,
            main_industry, floating_share_millions, market_cap_usd_millions, share_outstanding_millions,
            ipo_date, logo, name, phone, ticker, weburl, raw_json
        ) VALUES (
            %(symbol)s, %(country)s, %(currency)s, %(estimateCurrency)s,
            %(exchange)s, %(finnhubIndustry)s, %(floatingShare)s, %(marketCapitalization)s, %(shareOutstanding)s,
            %(ipo)s::date, %(logo)s, %(name)s, %(phone)s, %(ticker)s, %(weburl)s, %(raw_json)s
        )
        ON CONFLICT (symbol) DO UPDATE SET
            country=EXCLUDED.country, currency=EXCLUDED.currency, estimate_currency=EXCLUDED.estimate_currency,
            exchange=EXCLUDED.exchange, main_industry=EXCLUDED.main_industry,
            floating_share_millions=EXCLUDED.floating_share_millions,
            market_cap_usd_millions=EXCLUDED.market_cap_usd_millions,
            share_outstanding_millions=EXCLUDED.share_outstanding_millions,
            ipo_date=EXCLUDED.ipo_date, logo=EXCLUDED.logo, name=EXCLUDED.name,
            phone=EXCLUDED.phone, ticker=EXCLUDED.ticker, weburl=EXCLUDED.weburl,
            raw_json=EXCLUDED.raw_json;
        """, {
            "symbol": sym, "country": prof.get("country"), "currency": prof.get("currency"),
            "estimateCurrency": prof.get("estimateCurrency"), "exchange": prof.get("exchange"),
            "finnhubIndustry": prof.get("finnhubIndustry"), "floatingShare": prof.get("floatingShare"),
            "marketCapitalization": prof.get("marketCapitalization"), "shareOutstanding": prof.get("shareOutstanding"),
            "ipo": prof.get("ipo"), "logo": prof.get("logo"), "name": prof.get("name"),
            "phone": prof.get("phone"), "ticker": prof.get("ticker"), "weburl": prof.get("weburl"),
            "raw_json": psycopg2.extras.Json(prof)
        })

    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()
    ensure_table(cur)
    for sym, prof in data.items():
        if prof:  # Only upsert non-empty
            upsert(cur, sym, prof)
    conn.commit()
    cur.close()
    conn.close()
    log.info("Re-loaded all data to PostgreSQL")

# ----------------------------------------------------------------------
# Main
# ----------------------------------------------------------------------
def main() -> None:
    data, json_path = load_json()
    failures = find_failures(data)
    if not failures:
        log.info("No failures found – all good!")
        return

    save_failed_list(failures)

    new_profiles = {}
    for sym in failures:
        log.info(f"Refetching {sym}")
        prof = finnhub_profile(sym)
        new_profiles[sym] = prof
        if prof:
            log.info(f"Success: {sym} → {prof.get('name', 'No name')}")
        else:
            log.warning(f"Still failed: {sym}")
        time.sleep(REQUEST_DELAY)

    if UPDATE_JSON:
        update_json(data, json_path, new_profiles)

    if RELOAD_DB:
        reload_to_db(data)

    log.info("Refetch complete! Check failed_symbols.txt for any remaining issues.")

if __name__ == "__main__":
    main()

2025-10-30 12:33:05,276 | INFO | Loaded 503 profiles from C:\github\Bootcamp-Final-Project\python\sp500_finnhub_profiles.json
2025-10-30 12:33:05,278 | INFO | Found 20 failures: ['COR', 'CEG', 'COO', 'CPRT', 'FIS', 'FITB', 'GIS', 'HSIC', 'HST', 'ICE', 'INVH', 'JCI', 'JPM', 'K', 'KVUE', 'KDP', 'KEY', 'KEYS', 'PTC', 'VTRS']
2025-10-30 12:33:05,280 | INFO | Saved failures to failed_finnhub_profiles.txt
2025-10-30 12:33:05,282 | INFO | Refetching COR
2025-10-30 12:33:06,487 | INFO | Success: COR → Cencora Inc
2025-10-30 12:33:07,589 | INFO | Refetching CEG
2025-10-30 12:33:08,540 | INFO | Success: CEG → Constellation Energy Corp
2025-10-30 12:33:09,642 | INFO | Refetching COO
2025-10-30 12:33:10,586 | INFO | Success: COO → Cooper Companies Inc
2025-10-30 12:33:11,688 | INFO | Refetching CPRT
2025-10-30 12:33:12,626 | INFO | Success: CPRT → Copart Inc
2025-10-30 12:33:13,729 | INFO | Refetching FIS
2025-10-30 12:33:14,646 | INFO | Success: FIS → Fidelity National Information Services Inc
20